### *`Watermark Mask`*

In [157]:
import cv2

def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)
    img_canny = cv2.Canny(img_blur, 180, 60)
    img_dilate = cv2.dilate(img_canny, None, iterations=1)
    return cv2.erode(img_dilate, None, iterations=1)

def get_watermark(img):
    contours, _ = cv2.findContours(process(img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img.fill(255)
    for cnt in contours:
        if cv2.contourArea(cnt) > 100:
            cv2.drawContours(img, [cnt], -1, 0, -1)

img = cv2.imread("cwatermark.jpg")
get_watermark(img)
img = cv2.bitwise_not(img)
cv2.imshow("Watermark", img)
cv2.imwrite("mask.jpg", img)
cv2.waitKey(0)

-1

In [159]:
import cv2
import numpy as np

img = cv2.imread("input.jpg")
mask = cv2.imread('Fmask.jpg')
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
dilatekernel = np.ones((4, 4), 'uint8')
mask = cv2.dilate(mask, dilatekernel)
out = cv2.inpaint(img, mask, 6, flags= cv2.INPAINT_TELEA)
#cv2.imshow("Input", img)
#cv2.imshow("Watermark", mask)
cv2.imshow("Output", out)
cv2.imwrite("no_wm.jpg", out)
cv2.waitKey(0)

-1

In [161]:
import cv2
import numpy as np

# Read input image and mask
img = cv2.imread("input.jpg")
mask = cv2.imread('Fmask.jpg', cv2.IMREAD_GRAYSCALE)

# Dilate the mask
dilate_kernel = np.ones((7, 7), np.uint8)
mask_dilated = cv2.dilate(mask, dilate_kernel)

# Inpainting
out = cv2.inpaint(img, mask, 25, flags=cv2.INPAINT_TELEA)

# Apply filters to enhance the output
out = cv2.medianBlur(out, 9)  # Median filter
out = cv2.morphologyEx(out, cv2.MORPH_CLOSE, np.ones((9, 9), np.uint8))  # Closing operation

# Sharpening the inpainted region using unsharp masking
sharpness = 0.5 # Adjust the sharpness factor according to your preference
blurred = cv2.GaussianBlur(out, (0, 0), 3)
sharpened = cv2.addWeighted(out, 1. + sharpness, blurred, -sharpness, 0)

# Apply the sharpened region to the original image using the mask
result = img.copy()
result[mask_dilated != 0] = sharpened[mask_dilated != 0]

# Display output
cv2.imshow("Output", result)
cv2.imwrite("no_wm1.jpg", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [164]:
import cv2
import numpy as np

# Read input and mask images
img = cv2.imread("input.jpg")
mask = cv2.imread('mask.jpg', cv2.IMREAD_GRAYSCALE)

# Create a black canvas with the same dimensions as the input image
black_canvas = np.zeros_like(img)

# Calculate the position to place the mask at the center of the canvas
x_offset = (black_canvas.shape[1] - mask.shape[1]) // 2
y_offset = (black_canvas.shape[0] - mask.shape[0]) // 2

# Expand the dimensions of the mask to match the number of channels in the black canvas
expanded_mask = np.expand_dims(mask, axis=-1)
expanded_mask = np.repeat(expanded_mask, 3, axis=-1)

# Place the mask at the center of the black canvas
black_canvas[y_offset:y_offset+mask.shape[0], x_offset:x_offset+mask.shape[1]] = expanded_mask

# Convert the mask to 8-bit single-channel image
dilated_mask = cv2.cvtColor(black_canvas, cv2.COLOR_BGR2GRAY)

# Dilate the mask
dilatekernel = np.ones((4, 4), 'uint8')
dilated_mask = cv2.dilate(dilated_mask, dilatekernel)

# Inpaint the input image using the dilated mask
out = cv2.inpaint(img, dilated_mask, 7, flags=cv2.INPAINT_TELEA)

# Display the output image
cv2.imshow("Watermark", dilated_mask)
cv2.imshow("Output", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

### *`Watermark Removal`*

In [3]:
import cv2
import numpy as np
import os

def remove_watermark(input_path, output_path, mask_path):
    img = cv2.imread(input_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Create a black canvas with the same dimensions as the input image
    black_canvas = np.zeros_like(img)

    # Calculate the position to place the mask at the center of the canvas
    x_offset = (black_canvas.shape[1] - mask.shape[1]) // 2
    y_offset = (black_canvas.shape[0] - mask.shape[0]) // 2

    # Expand the dimensions of the mask to match the number of channels in the black canvas
    expanded_mask = np.expand_dims(mask, axis=-1)
    expanded_mask = np.repeat(expanded_mask, 3, axis=-1)

    # Place the mask at the center of the black canvas
    black_canvas[y_offset:y_offset+mask.shape[0], x_offset:x_offset+mask.shape[1]] = expanded_mask

    # Convert the mask to 8-bit single-channel image
    dilated_mask = cv2.cvtColor(black_canvas, cv2.COLOR_BGR2GRAY)

    # Dilate the mask
    dilatekernel = np.ones((4, 4), 'uint8')
    dilated_mask = cv2.dilate(dilated_mask, dilatekernel)

    # Inpaint the input image using the dilated mask
    out = cv2.inpaint(img, dilated_mask, 7, flags=cv2.INPAINT_TELEA)
    cv2.imwrite(output_path, out)

# Directory paths
old_dir = "old"
cleaned_dir = "cleaned_images"
mask_path = "mask.jpg"

# Ensure cleaned images directory exists
if not os.path.exists(cleaned_dir):
    os.makedirs(cleaned_dir)

# Iterate over each image in the old directory
for filename in os.listdir(old_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        input_path = os.path.join(old_dir, filename)
        output_path = os.path.join(cleaned_dir, filename)
        remove_watermark(input_path, output_path, mask_path)

### *`Others`*

In [10]:
import cv2

def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply adaptive thresholding
    _, img_thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    # Perform morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_open = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    img_close = cv2.morphologyEx(img_open, cv2.MORPH_CLOSE, kernel, iterations=2)
    return img_close

def get_watermark(img):
    contours, _ = cv2.findContours(process(img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img.fill(255)
    for cnt in contours:
        # Refine contour filtering based on area and other properties
        if cv2.contourArea(cnt) > 100:
            cv2.drawContours(img, [cnt], -1, 0, -1)

img = cv2.imread("watermark.jpg")
get_watermark(img)
# Inpainting to fill the removed regions
img = cv2.inpaint(img, (img == 0).astype(int), inpaintRadius=3, flags=cv2.INPAINT_TELEA)
cv2.imshow("Watermark Removed", img)
cv2.waitKey(0)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2
import numpy as np

def remove_watermark(img, template_path):
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    if template is None:
        print("Error: Template image not found.")
        return img

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    h, w = template.shape[::-1]  # Get width and height of the template

    res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    loc = np.where(res >= threshold)

    # Debugging: Visualize matched regions
    img_debug = img.copy()
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img_debug, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)

    cv2.imshow("Debug: Matched Regions", img_debug)
    cv2.waitKey(0)w
    cv2.destroyAllWindows()

    return img

img = cv2.imread("input.jpg")
if img is None:
    print("Error: Input image not found.")
else:
    template_path = "watermark.jpg"
    result = remove_watermark(img, template_path)

    cv2.imshow("Watermark Removed", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

img = cv2.imread("input.jpg")
img2 = cv2.imread('Fwatermark.jpg')
# remove watermark with mark
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
dilatekernel = np.ones((5, 5), 'uint8')
img2 = cv2.dilate(img2, dilatekernel)
out = cv2.inpaint(img, img2, 3, flags= cv2.INPAINT_NS)
cv2.imshow("Input", img)
cv2.waitKey(0)
cv2.imshow("Watermark", img2)
cv2.waitKey(0)
cv2.imshow("Output", out)
cv2.waitKey(0)

-1

In [ ]:
import cv2
import numpy as np

def invert_mask(mask):
    # Invert the mask by bitwise not operation
    inverted_mask = cv2.bitwise_not(mask)
    return inverted_mask

# Load the mask
mask = cv2.imread('watermark.jpg', 0)  # Read mask as grayscale

# Invert the mask
inverted_mask = invert_mask(mask)

# Display or save the inverted mask
cv2.imshow('Inverted Mask', inverted_mask)
cv2.imwrite("Fwatermark.jpg", inverted_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def remove_watermark(image_path, mask_path):
    # Load the image and the mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Convert the mask to binary (if needed)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Apply inpainting
    result = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    return result

# Example usage
image_path = 'input.jpg'
mask_path = 'Fwatermark.jpg'
output_image = remove_watermark(image_path, mask_path)
cv2.imshow('Output Image', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2

def remove_watermark(image_path, mask_path):
    # Load the image and the mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to the mask to get a binary mask
    _, binary_mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)

    # Invert the binary mask
    inverted_mask = cv2.bitwise_not(binary_mask)

    # Apply inpainting to the grayscale image using the inverted mask
    inpainted_image = cv2.inpaint(gray_image, inverted_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    # Convert the inpainted image back to BGR
    inpainted_image_bgr = cv2.cvtColor(inpainted_image, cv2.COLOR_GRAY2BGR)

    return inpainted_image_bgr

# Example usage
image_path = 'image_with_watermark.jpg'
mask_path = 'watermark.png'
output_image = remove_watermark(image_path, mask_path)
cv2.imwrite('image_without_watermark.jpg', output_image)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2
import numpy as np
import os

directory = "old/"
image_files = os.listdir(directory)

# Create a folder for the processed images (if it doesn't exist)
output_folder = "cleaned_images"
os.makedirs(output_folder, exist_ok=True)

def process_image(img):
    
    return new_img

# Create a window to display images
cv2.namedWindow('Images', cv2.WINDOW_NORMAL)

# Iterate over each image file
for image_file in image_files:
    # Read the image
    img = cv2.imread(os.path.join(directory, image_file))

    # Process the image
    processed_img = process_image(img)

    # Concatenate the original and processed images horizontally
    combined_image = np.hstack((img, processed_img))

    # Display the combined image
    cv2.imshow('Images', combined_image)
    cv2.waitKey(0)  # Wait indefinitely until a key is pressed

# Close the window after all images are displayed
cv2.destroyAllWindows()

In [ ]:
import cv2
import os
import numpy as np
from PIL import ImageTk, Image
import tkinter as tk
from tkinter import filedialog, Button, messagebox

# Function to open an image with OpenCV and display it in a Tkinter window
def open_img():
    global img_with_watermark, img_without_watermark
    filename = filedialog.askopenfilename(title="Select file")

    img_with_watermark = cv2.imread(filename, 1)
    cv2.imshow("Image With Watermark", img_with_watermark)

    img_without_watermark = cv2.imread(filename)
    _, thresh = cv2.threshold(img_without_watermark, 150, 255, cv2.THRESH_BINARY)
    cv2.imshow('Image Without Watermark', thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function to exit the application
def exit_win():
    if messagebox.askokcancel("Exit", "Do you want to exit?"):
        window.destroy()

# Tkinter window setup
window = tk.Tk()
window.title("Image Watermark Remover")
window.geometry('1000x700')

# Top label
start1 = tk.Label(text="Image Watermark Remover", font=("Arial", 50), fg="magenta")
start1.place(x=80, y=10)

# Image on the main window
path = "old/1.jpg"
img1 = ImageTk.PhotoImage(Image.open(path))
panel = tk.Label(window, image=img1)
panel.image = img1  # Retain reference to the image object
panel.place(x=150, y=120)

# Second image label
sec1 = tk.Label(text="Select any image with Watermark & Remove it...", font=("Arial", 40), fg="green")
sec1.place(x=200, y=350)

# Select Button
selectb = Button(window, text="SELECT", command=open_img, font=("Arial", 25), bg="light green", fg="blue")
selectb.place(x=150, y=550)

# Exit Button
exitb = Button(window, text="EXIT", command=exit_win, font=("Arial", 25), bg="red", fg="blue")
exitb.place(x=700, y=550)

window.protocol("WM_DELETE_WINDOW", exit_win)
window.mainloop()


TclError: image "pyimage3" doesn't exist